In [ ]:
# importing libraries
import numpy as np
import networkx as nx
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Create a sample graph
G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)])  # Adding connections

# Assign sample text data to nodes
G.nodes[1]['text'] = "This product is amazing!"
G.nodes[2]['text'] = "this is bad."
G.nodes[3]['text'] = "It's not great."
G.nodes[4]['text'] = "I love it! Highly recommend."
G.nodes[5]['text'] = "i hate it."
G.nodes[6]['text'] = "everything is awesome."

In [ ]:
#pre processing data

# Extract text data from nodes
texts = [G.nodes[n]['text'] for n in G.nodes]

max_words = 5000

# Tokenization
tokenizer = Tokenizer(num_words = max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding sequences
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")


In [ ]:
#Build & Train an RNN (LSTM Model)python

# Define model parameters
embedding_dim = 16
output_dim = 1  # Binary classification (positive/negative)

# Create RNN Model
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(16, activation="relu"),
    Dense(output_dim, activation="sigmoid")])  # Sigmoid for binary classification

# Compile Model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Dummy labels (1 for positive, 0 for negative)
import numpy as np
labels = [1, 0, 0, 1, 0, 1]
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

# Train Model
model.fit(padded_sequences, labels, epochs=20)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6667 - loss: 0.6931
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5000 - loss: 0.6926
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.6667 - loss: 0.6921
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 1.0000 - loss: 0.6916
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.8333 - loss: 0.6911
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.6667 - loss: 0.6906
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.6667 - loss: 0.6900
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.6667 - loss: 0.6892
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.8333 - loss: 0.6884
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.8333 - loss: 0.6874
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 1.0000 - loss: 0.6862
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 1.0000 - loss: 0.684

In [ ]:
# Predict sentiment scores for each node
sentiment_scores = model.predict(padded_sequences)
for i, node in enumerate(G.nodes):
    G.nodes[node]['sentiment'] = float(sentiment_scores[i])  # Assigning sentiment score


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step


<ipython-input-35-1165e659d541>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  G.nodes[node]['sentiment'] = float(sentiment_scores[i])  # Assigning sentiment score


In [ ]:
# Analyze sentiment-based communities
positive_nodes = [n for n in G.nodes if G.nodes[n]['sentiment'] > 0.5]
negative_nodes = [n for n in G.nodes if G.nodes[n]['sentiment'] <= 0.5]

print("Positive Sentiment Nodes:", positive_nodes)
print("Negative Sentiment Nodes:", negative_nodes)

Positive Sentiment Nodes: [1, 4, 6]
Negative Sentiment Nodes: [2, 3, 5]


In [1]:
# Explaination

In [ ]:
'''
1. Import Libraries:
    NumPy: Used for handling arrays and numerical operations.

    NetworkX: Used for creating and analyzing graph structures.

    TensorFlow: A deep learning library to build and train the LSTM model.

    Tokenizer & Pad Sequences: From Keras, used for text preprocessing.

2. Create a Graph:
    A graph G is created using networkx.Graph(). This graph represents a collection of nodes and edges.

    Edges are added using G.add_edges_from(). In this case, nodes 1 through 6 are connected in a linear fashion.

    Example:

    (1) -- (2) -- (3) -- (4) -- (5) -- (6)
3. Assign Sample Text Data to Nodes:
    Each node (from 1 to 6) is assigned a piece of text that represents some opinion or sentiment about a product.

    The text data will be used for sentiment analysis to classify the opinions as positive or negative.

4. Text Preprocessing:
    Extracting Text Data: A list of text from all nodes is created.

    Tokenization: Tokenizer from Keras is used to convert the text data into numerical sequences. 
    This step transforms each word into a unique integer based on its frequency in the dataset.

    Padding Sequences: The sequences are padded to ensure that all inputs to the model have the same length. 
    This is done by pad_sequences(), ensuring that shorter sequences are padded with zeros at the end.

5. Build & Train the RNN Model:
    Model Architecture:

    Embedding Layer: Converts input words into dense vectors of fixed size (16 here). The words are represented in a continuous vector space where similar words have similar vector representations.

    LSTM Layers: The Long Short-Term Memory (LSTM) layers are used to capture dependencies in the sequence of words. There are two LSTM layers:

    The first LSTM layer returns sequences of outputs (to feed into the next LSTM layer).

    The second LSTM layer outputs a single vector (for further dense layers).

    Dense Layers: The model has dense layers that perform computations after the LSTM layers, and finally, a sigmoid activation function is used to output a probability (0 to 1) for binary classification.

    Compiling the Model:

    Loss Function: binary_crossentropy is used because we are dealing with binary classification (positive/negative sentiment).

    Optimizer: adam, a popular optimizer for training deep learning models.

    Metrics: accuracy to measure the model's performance.

    Training the Model: The model is trained using the text data (padded sequences) and labels (0 for negative, 1 for positive sentiment). It is trained for 20 epochs.

6. Prediction and Sentiment Assignment:
    Prediction: After training, the model predicts sentiment scores (between 0 and 1) for each node's text. This score represents the sentiment of the text assigned to the node.

    Assign Sentiment to Nodes: The predicted sentiment score for each node is stored as an attribute in the graph. Nodes with sentiment scores greater than 0.5 are classified as "positive" sentiment, while nodes with scores 0.5 or lower are classified as "negative".

7. Sentiment-Based Community Analysis:
    Positive Nodes: All nodes with a sentiment score greater than 0.5 are classified as having a positive sentiment.

    Negative Nodes: All nodes with a sentiment score less than or equal to 0.5 are classified as having a negative sentiment.

    The positive and negative nodes are then printed out.

    Output:
    The code prints the nodes of the graph that have positive and negative sentiment based on their sentiment scores. For example:

    Positive Sentiment Nodes: [1, 4, 6]
    Negative Sentiment Nodes: [2, 3, 5]

'''